<a href="https://colab.research.google.com/github/Keiji2827/data_scientist/blob/master/Chapter6_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6 Pandasを使ったデータ加工処理

- **[6.1 Pandas](#6.1-Pandas)**
    - [6.1.1 階層型インデックス](#6.1.1-階層型インデックス)
    - [6.1.2 データのマージ](#6.1.2-データのマージ)
    - [6.1.3 データの操作と変換](#6.1.3-データの操作と変換)
    - [6.1.4 データの集約とグループ演算](#6.1.4-データの集約とグループ演算)
<br><br>
- **[6.2 欠損データと異常値の取り扱いの基礎](#6.2-欠損データと異常値の取り扱いの基礎)**
    - [6.2.1 欠損データの扱い方](#6.2.1-欠損データの扱い方)
    - [6.2.2 異常データの扱い方](#6.2.2-異常データの扱い方)
<br><br>
- **[6.3 総合問題](#6.3-総合問題)**
    - [6.3.1 総合問題1](#6.3.1-総合問題1)

***

## 6.1 Pandas
ゴール：Pandasの階層型インデックスを使える、データの結合ができる、group byなどを使って集計処理ができる

In [0]:
# 以下のモジュールを使うので、あらかじめ読み込んでおいてください
import numpy as np
import numpy.random as random
import scipy as sp
import pandas as pd
from pandas import Series, DataFrame

# 可視化モジュール
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 小数第３まで表示
%precision 3

'%.3f'

### 6.1.1 階層型インデックス
キーワード：階層型インデックス


この章では、データを自由にハンドリングするためのPandasの応用的な処理について学んでいきます。まずは、**階層型インデックス**です。以前Pandasのインデックスについて少し扱いましたが、1つのインデックスだけではなく、階層的に設定したいこともあります。階層的に設定することで、各階層ごとに集計や計算が可能になり、後々便利です。

以下のデータセットでは、インデックスを2段構造で設定しており、indexでその値を設定します。はじめがaとb、次が1と2でインデックスを設定しています。

In [0]:
hier_data_frame = DataFrame(np.arange(9).reshape((3,3))
                           ,index = [['a','a','b'],[1,2,2]]
                           ,columns = [['Osaka','Tokyo','Osaka']
                                      ,['Blue','Red','Red']]
                           )
hier_data_frame

Osaka Tokyo Osaka
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8

先ほど設定したインデックスに名前をつけることも可能です。

In [0]:
# indexに名前を付ける
hier_data_frame.index.names =['key1','key2']
# カラムに名前を付ける
hier_data_frame.columns.names =['city','color']
hier_data_frame

city      Osaka Tokyo Osaka
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5
b    2        6     7     8

ここで例えば、カラムのcityについて、「Osaka」のデータだけ見たいとしましょう。グループの絞り込みで、データを抽出しています。

In [0]:
hier_data_frame['Osaka']

color      Blue  Red
key1 key2           
a    1        0    2
     2        3    5
b    2        6    8

次はあるインデックスを軸にした集計で、以下はkey2ごとに合計を計算しています。

In [0]:
# 階層ごとの要約統計量：行合計
hier_data_frame.sum(level='key2')

city,Osaka,Tokyo,Osaka
color,Blue,Red,Red
key2,,,
1,0,1,2
2,9,11,13


次は、「color」ごとの合計値です。軸はaxisで設定しています。

In [0]:
# 列合計
hier_data_frame.sum(level='color',axis=1)

color      Blue  Red
key1 key2           
a    1        0    3
     2        3    9
b    2        6   15

なお、あるインデックスを削除したい場合は、dropを使うことでインデックスの要素を削除できます。以下は　key1のbを削除しています。

In [0]:
hier_data_frame.drop(["b"])

city      Osaka Tokyo Osaka
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5

以上で、階層型インデックスの説明を終わります。

#### <練習問題 1>

次のデータに対して、Kyotoの列だけ抜き出してみましょう。

In [0]:
hier_data_frame1 = DataFrame(np.arange(12).reshape((3,4))
                           ,index = [['c','d','d'],[1,2,1]]
                           ,columns = [['Kyoto','Nagoya','Hokkaido','Kyoto']
                                      ,['Yellow','Yellow','Red','Blue']]
                           )

hier_data_frame1.index.names =['key1','key2']
hier_data_frame1.columns.names =['city','color']
hier_data_frame1

city       Kyoto Nagoya Hokkaido Kyoto
color     Yellow Yellow      Red  Blue
key1 key2                             
c    1         0      1        2     3
d    2         4      5        6     7
     1         8      9       10    11

#### <練習問題 2>

練習問題1のデータに対して、cityをまとめて列同士の平均値を出してください。

#### <練習問題 3>

練習問題1のデータに対して、key2ごとに行の合計値を算出してみましょう。


### 6.1.2 データのマージ
キーワード：内部結合、外部結合、横結合、縦結合

データの結合については前の章で少し学びました。データを結合したいケースは多々あり、データをつなぐことで集計がしやすくなったり、新しい軸における値がわかったりするので、使えるようになりましょう。ただ、結合と言っても色々なパターンがあります。以下でそれらを紹介していきます。

まずは、ここで学ぶマージの対象となるデータを準備します。後の章で学ぶデータベース（SQL）でも同じようなデータ操作（結合）を学びます。

In [0]:
# データ1の準備
attri_data1 = {'ID':['100','101','102','103','104','106','108','110','111','113']
        ,'city':['Tokyo','Osaka','Kyoto','Hokkaido','Tokyo','Tokyo','Osaka','Kyoto','Hokkaido','Tokyo']
        ,'birth_year':[1990,1989,1992,1997,1982,1991,1988,1990,1995,1981]
        ,'name':['Hiroshi','Akiko','Yuki','Satoru','Steeve','Mituru','Aoi','Tarou','Suguru','Mitsuo']}
attri_data_frame1 = DataFrame(attri_data1)
attri_data_frame_index1 = DataFrame(attri_data1,index=['e','b','a','d','c','f','g','h','j','i'])
attri_data_frame_index1

,ID,birth_year,city,name
e,100,1990,Tokyo,Hiroshi
b,101,1989,Osaka,Akiko
a,102,1992,Kyoto,Yuki
d,103,1997,Hokkaido,Satoru
c,104,1982,Tokyo,Steeve
f,106,1991,Tokyo,Mituru
g,108,1988,Osaka,Aoi
h,110,1990,Kyoto,Tarou
j,111,1995,Hokkaido,Suguru
i,113,1981,Tokyo,Mitsuo


In [0]:
# データ2の準備
attri_data2 = {'ID':['100','101','102','105','107']
        ,'math':[50,43,33,76,98]
        ,'English':[90,30,20,50,30]
        ,'sex':['M','F','F','M','M']
        ,'index_num':[0,1,2,3,4]}
attri_data_frame2 = DataFrame(attri_data2)
attri_data_frame2

,English,ID,index_num,math,sex
0,90,100,0,50,M
1,30,101,1,43,F
2,20,102,2,33,F
3,50,105,3,76,M
4,30,107,4,98,M


次に、先ほどのデータ1と、このデータ2を結合するのですが、以下の4パターンが考えられます。左上図が内部結合といって両方にキーがある場合に結合します。右上図が全結合といい特にどちらかにキーがある場合に結合します。左下図が左外部結合といい左側にあるデータのキーがある時に結合され、右下図が右外部結合といい左結合の右バージョンになります。

ここで使うのは主に内部結合と（左）外部結合なので、この2つは理解しておいてください。

![comment](http://www.dofactory.com/Images/sql-joins.png)
参照URL:http://www.dofactory.com/Images/sql-joins.png

上記のデータ2つに対して、IDをキーとして結合すると、以下のようになります。onをキーとして設定します。

In [0]:
# データのマージ（内部結合、inner　join が省略されてる、またキーは自動的に認識されるが、onで明示的に指定可能）
# また複数キーも可能、リストで指定
print("・結合テーブル")
pd.merge(attri_data_frame1,attri_data_frame2,on='ID')

・結合テーブル


,ID,birth_year,city,name,English,index_num,math,sex
0,100,1990,Tokyo,Hiroshi,90,0,50,M
1,101,1989,Osaka,Akiko,30,1,43,F
2,102,1992,Kyoto,Yuki,20,2,33,F


両方ともキーがあるレコードだけ表示されました。

次は、左側のテーブルに合わせて、データフレーム2のデータを結合しています。左側に対応するデータが右にない場合は、NaNになります。これが左外部結合です。

In [0]:
# データのマージ（left）
pd.merge(attri_data_frame1,attri_data_frame2,how='left')

,ID,birth_year,city,name,English,index_num,math,sex
0,100,1990,Tokyo,Hiroshi,90.0,0.0,50.0,M
1,101,1989,Osaka,Akiko,30.0,1.0,43.0,F
2,102,1992,Kyoto,Yuki,20.0,2.0,33.0,F
3,103,1997,Hokkaido,Satoru,NaN,NaN,NaN,NaN
4,104,1982,Tokyo,Steeve,NaN,NaN,NaN,NaN
5,106,1991,Tokyo,Mituru,NaN,NaN,NaN,NaN
6,108,1988,Osaka,Aoi,NaN,NaN,NaN,NaN
7,110,1990,Kyoto,Tarou,NaN,NaN,NaN,NaN
8,111,1995,Hokkaido,Suguru,NaN,NaN,NaN,NaN
9,113,1981,Tokyo,Mitsuo,NaN,NaN,NaN,NaN


次は、どちらのデータにも存在するデータで結合しています。これが全結合です。値がない場合は、NaNになります。

In [0]:
# データのマージ（outer）
pd.merge(attri_data_frame1,attri_data_frame2,how='outer')

,ID,birth_year,city,name,English,index_num,math,sex
0,100,1990.0,Tokyo,Hiroshi,90.0,0.0,50.0,M
1,101,1989.0,Osaka,Akiko,30.0,1.0,43.0,F
2,102,1992.0,Kyoto,Yuki,20.0,2.0,33.0,F
3,103,1997.0,Hokkaido,Satoru,NaN,NaN,NaN,NaN
4,104,1982.0,Tokyo,Steeve,NaN,NaN,NaN,NaN
5,106,1991.0,Tokyo,Mituru,NaN,NaN,NaN,NaN
6,108,1988.0,Osaka,Aoi,NaN,NaN,NaN,NaN
7,110,1990.0,Kyoto,Tarou,NaN,NaN,NaN,NaN
8,111,1995.0,Hokkaido,Suguru,NaN,NaN,NaN,NaN
9,113,1981.0,Tokyo,Mitsuo,NaN,NaN,NaN,NaN


なお、キーをインデックスで指定して結合することも可能です。

In [0]:
# index によるマージ
pd.merge(attri_data_frame1,attri_data_frame2,left_index=True,right_on='index_num')

,ID_x,birth_year,city,name,English,ID_y,index_num,math,sex
0,100,1990,Tokyo,Hiroshi,90,100,0,50,M
1,101,1989,Osaka,Akiko,30,101,1,43,F
2,102,1992,Kyoto,Yuki,20,102,2,33,F
3,103,1997,Hokkaido,Satoru,50,105,3,76,M
4,104,1982,Tokyo,Steeve,30,107,4,98,M


次は、concatを使ってデータの縦結合をしています。これまでは、何らかのキーに紐付いてデータをマージしていましたが、次の処理ではそのまま縦に積み上げています。

In [0]:
# データの準備
attri_data3 = {'ID':['117','118','119','120','125']
        ,'city':['Chiba','Kanagawa','Tokyo','Fukuoka','Okinawa']
        ,'birth_year':[1990,1989,1992,1997,1982]
        ,'name':['Suguru','Kouichi','Satochi','Yukie','Akari']}
attri_data_frame3 = DataFrame(attri_data3)


In [0]:
# concat 縦結合
concat_data = pd.concat([attri_data_frame1,attri_data_frame3])
# 注意：カラムがないとNaNになる
concat_data

,ID,birth_year,city,name
0,100,1990,Tokyo,Hiroshi
1,101,1989,Osaka,Akiko
2,102,1992,Kyoto,Yuki
3,103,1997,Hokkaido,Satoru
4,104,1982,Tokyo,Steeve
5,106,1991,Tokyo,Mituru
6,108,1988,Osaka,Aoi
7,110,1990,Kyoto,Tarou
8,111,1995,Hokkaido,Suguru
9,113,1981,Tokyo,Mitsuo


#### <練習問題 1>

下記の2つのデータテーブルに対して、内部結合してみましょう。

In [0]:
# データ4の準備
attri_data4 = {'ID':['0','1','2','3','4','6','8','11','12','13']
        ,'city':['Tokyo','Osaka','Kyoto','Hokkaido','Tokyo','Tokyo','Osaka','Kyoto','Hokkaido','Tokyo']
        ,'birth_year':[1990,1989,1992,1997,1982,1991,1988,1990,1995,1981]
        ,'name':['Hiroshi','Akiko','Yuki','Satoru','Steeve','Mituru','Aoi','Tarou','Suguru','Mitsuo']}
attri_data_frame4 = DataFrame(attri_data4)
attri_data_frame4

,ID,birth_year,city,name
0,0,1990,Tokyo,Hiroshi
1,1,1989,Osaka,Akiko
2,2,1992,Kyoto,Yuki
3,3,1997,Hokkaido,Satoru
4,4,1982,Tokyo,Steeve
5,6,1991,Tokyo,Mituru
6,8,1988,Osaka,Aoi
7,11,1990,Kyoto,Tarou
8,12,1995,Hokkaido,Suguru
9,13,1981,Tokyo,Mitsuo


In [0]:
# データ5の準備
attri_data5 = {'ID':['0','1','3','6','8']
        ,'math':[20,30,50,70,90]
        ,'English':[30,50,50,70,20]
        ,'sex':['M','F','F','M','M']
        ,'index_num':[0,1,2,3,4]}
attri_data_frame5 = DataFrame(attri_data5)
attri_data_frame5

,English,ID,index_num,math,sex
0,30,0,0,20,M
1,50,1,1,30,F
2,50,3,2,50,F
3,70,6,3,70,M
4,20,8,4,90,M


#### <練習問題 2>

attri_data_frame4をベースにattri_data_frame5のテーブルを外部結合してみましょう。

#### <練習問題 3>
attri_data_frame4に対して、以下のデータを縦結合してみましょう。

In [0]:
# データの準備
attri_data6 = {'ID':['70','80','90','120','150']
        ,'city':['Chiba','Kanagawa','Tokyo','Fukuoka','Okinawa']
        ,'birth_year':[1980,1999,1995,1994,1994]
        ,'name':['Suguru','Kouichi','Satochi','Yukie','Akari']}
attri_data_frame6 = DataFrame(attri_data6)

### 6.1.3 データの操作と変換
キーワード：データのピボット操作、重複データ、マッピング、ビン分割

次は、データの操作と変換（ピボット操作、データの重複があった場合の処理、マッピング、ビン分割など）について扱っていきましょう。

まずは、データのピボット操作について学びます。先ほどの階層テーブルhier_data_frameを使って、以下の処理でデータフレームの再構成ができます。stackで列にあったものが行にきます。

In [0]:
#　ピボット　列が行に 
hier_data_frame.stack()

city             Osaka  Tokyo
key1 key2 color              
a    1    Blue       0    NaN
          Red        2    1.0
     2    Blue       3    NaN
          Red        5    4.0
b    2    Blue       6    NaN
          Red        8    7.0

一方、unstackを使うことで逆の操作が可能です。

In [0]:
# 再配置
hier_data_frame.stack().unstack()

city      Osaka     Tokyo     
color      Blue Red  Blue  Red
key1 key2                     
a    1        0   2   NaN  1.0
     2        3   5   NaN  4.0
b    2        6   8   NaN  7.0

上記のデータ操作では、縦にあったものを横に持ってきたり、横にあったものを縦に持ってきたりしており、これらのテクニックは、データのモデリング前の処理として使うことも多く便利ですので、ぜひ理解して使えるようになってください。

次は、重複があるデータの処理です。データ分析をやっていると、データに重複があることもありますし、自分で実際に集計等していて重複が混じることもあり、そのチェックをするという意味で重要です。

まず、重複するデータを準備しましょう。

In [0]:
#　重複データ
dupli_data = DataFrame({'col1':[1,1,2,3,4,4,6,6]
                       ,'col2':['a','b','b','b','c','c','b','b']})
print("・元のデータ")
dupli_data

・元のデータ


,col1,col2
0,1,a
1,1,b
2,2,b
3,3,b
4,4,c
5,4,c
6,6,b
7,6,b


重複の判定はdublicated()を使います。それぞれの行をみて、重複がある場合は、Trueと表示されます。

In [0]:
#　重複判定
print("・重複ありの行")
dupli_data.duplicated()

・重複ありの行


0    False
1    False
2    False
3    False
4    False
5     True
6    False
7     True
dtype: bool

drop_duplicates()で重複したデータを削除後のデータを表示します。

In [0]:
#　重複削除
print("・重複削除後のデータ")
dupli_data.drop_duplicates()

・重複削除後のデータ


,col1,col2
0,1,a
1,1,b
2,2,b
3,3,b
4,4,c
6,6,b


次は、マッピング処理です。これは、Excelの関数のvlookupのような処理です。共通のキーとなるデータに対して、一方の（参照）テーブルからそのキーに対応するデータをもってきます。以下は、都道府県名と地域名を対応付けた参照データです。東京は関東、京都と大阪は関西、北海道は北海道と変換しましょう。

In [0]:
# 参照データ
city_map ={'Tokyo':'Kanto'
          ,'Hokkaido':'Hokkaido'
          ,'Osaka':'Kansai'
          ,'Kyoto':'Kansai'}
city_map

{'Hokkaido': 'Hokkaido',
 'Kyoto': 'Kansai',
 'Osaka': 'Kansai',
 'Tokyo': 'Kanto'}

はじめに用意したattri_data_frame1のcityカラムをベースとして、上の参照データに対応する地域名データを持ってきて、新しいカラムを追加しています。

In [0]:
#　参照データを結合
# もし対応するデータがなかったら、NANになる。
attri_data_frame1['region'] = attri_data_frame1['city'].map(city_map)
attri_data_frame1

,ID,birth_year,city,name,region
0,100,1990,Tokyo,Hiroshi,Kanto
1,101,1989,Osaka,Akiko,Kansai
2,102,1992,Kyoto,Yuki,Kansai
3,103,1997,Hokkaido,Satoru,Hokkaido
4,104,1982,Tokyo,Steeve,Kanto
5,106,1991,Tokyo,Mituru,Kanto
6,108,1988,Osaka,Aoi,Kansai
7,110,1990,Kyoto,Tarou,Kansai
8,111,1995,Hokkaido,Suguru,Hokkaido
9,113,1981,Tokyo,Mitsuo,Kanto


新しい変数regionをつけることで、この単位で集計が可能になります。

次は、Pythonの基礎で学んだ無名関数とmapを使って、カラムの中の一部のデータを取り出す処理をしています。具体的には、birth_yearの上3桁を取得します。関数適応やループなどを使って要素を1つ1つ取り出して処理するより断然早いので、まとめて処理したい場合は、このようなやり方を検討することをおすすめします。

In [0]:
#　birth_year の上3つの数字・文字を取り出す
attri_data_frame1['up_two_num'] = attri_data_frame1['birth_year'].map(lambda x:str(x)[0:3])
attri_data_frame1

,ID,birth_year,city,name,region,up_two_num
0,100,1990,Tokyo,Hiroshi,Kanto,199
1,101,1989,Osaka,Akiko,Kansai,198
2,102,1992,Kyoto,Yuki,Kansai,199
3,103,1997,Hokkaido,Satoru,Hokkaido,199
4,104,1982,Tokyo,Steeve,Kanto,198
5,106,1991,Tokyo,Mituru,Kanto,199
6,108,1988,Osaka,Aoi,Kansai,198
7,110,1990,Kyoto,Tarou,Kansai,199
8,111,1995,Hokkaido,Suguru,Hokkaido,199
9,113,1981,Tokyo,Mitsuo,Kanto,198


最後にビン分割です。これは、ある離散的な範囲にデータを分割して集計したい場合に、便利な機能です。具体的には、上のデータのbirth_yearに対して、5年区切りで集計をしたい場合や、ある特定の分割で計算をしたい場合に、以下のようにビン分割したリストを用意し、pandasのcut関数を使って、処理しています。

In [0]:
#　分割の粒度
birth_year_bins = [1980,1985,1990,1995,2000]

# ビン分割の実施
birth_year_cut_data = pd.cut(attri_data_frame1.birth_year,birth_year_bins)
birth_year_cut_data

0    (1985, 1990]
1    (1985, 1990]
2    (1990, 1995]
3    (1995, 2000]
4    (1980, 1985]
5    (1990, 1995]
6    (1985, 1990]
7    (1985, 1990]
8    (1990, 1995]
9    (1980, 1985]
Name: birth_year, dtype: category
Categories (4, interval[int64]): [(1980, 1985] < (1985, 1990] < (1990, 1995] < (1995, 2000]]

上記の結果を使いそれぞれの数を集計したい場合は、value_countsを使います。

In [0]:
# 集計結果
pd.value_counts(birth_year_cut_data)

(1985, 1990]    4
(1990, 1995]    3
(1980, 1985]    2
(1995, 2000]    1
Name: birth_year, dtype: int64

それぞれのビンに名前をつけることも可能です。

In [0]:
# 名前付き
group_names = ["first1980","second1980","first1990","second1990"]
birth_year_cut_data = pd.cut(attri_data_frame1.birth_year,birth_year_bins,labels = group_names)
pd.value_counts(birth_year_cut_data)

second1980    4
first1990     3
first1980     2
second1990    1
Name: birth_year, dtype: int64

上記では、ビン分割のリストを用意しましたが、あらかじめ分割数を指定したい場合は、以下のように設定可能です。

In [0]:
# 数字で分割数指定可能。
# ここでは2つに分割
pd.cut(attri_data_frame1.birth_year,2)

0      (1989.0, 1997.0]
1    (1980.984, 1989.0]
2      (1989.0, 1997.0]
3      (1989.0, 1997.0]
4    (1980.984, 1989.0]
5      (1989.0, 1997.0]
6    (1980.984, 1989.0]
7      (1989.0, 1997.0]
8      (1989.0, 1997.0]
9    (1980.984, 1989.0]
Name: birth_year, dtype: category
Categories (2, interval[float64]): [(1980.984, 1989.0] < (1989.0, 1997.0]]

分位点での分割も可能で、qcutを使います。これを使うことで、ほぼ同じサイズのビンを作成することができます。

In [0]:
pd.value_counts(pd.qcut(attri_data_frame1.birth_year,2))

(1980.999, 1990.0]    6
(1990.0, 1997.0]      4
Name: birth_year, dtype: int64

このビン分割は、具体的には、顧客の購買金額合計を分けて分析をしたい場合など、マーケティング分析にも使えます。詳しいことは、総合問題演習で扱っていくことにしましょう。

#### <練習問題 1>

以前の章で使用した「student-mat.csv」のデータを使います。ageを2倍にしたカラムを追加してみましょう。

In [0]:
# chapter1で用意したデータがあるpathに移動して、以下を実行してください。例） cd pathの名前
student_data_math = pd.read_csv("student-mat.csv",sep=";")

#### <練習問題 2>

上記と同じデータで、「absences」のカラムについて、以下の3つのビンに分けてそれぞれの人数を数えてみましょう。なお、cutのオプション設定で、デフォルトは右側が閉区間になっていますが、今回は0が入るためのright=Falseを追加してください。

In [0]:
#　分割の粒度
absences_bins = [0,1,5,100]

#### <練習問題 3>

上記と同じデータで、「absences」のカラムについて、qcutを用いて3つのビンに分けてみましょう。

### 6.1.4 データの集約とグループ演算
キーワード：group by

ここでは、あるカラムを軸にして、集計する処理を学びます。以前の章で少し扱いましたが、group byを使うことで、ある変数を軸として、その単位で集計処理をします。以下は、cityをベースにそれぞれの数量を計算しています、

In [0]:
# サイズ情報
attri_data_frame1.groupby("city").size()

city
Hokkaido    2
Kyoto       2
Osaka       2
Tokyo       4
dtype: int64

次は、cityを軸に、birth_yearの平均値を算出しています。

In [0]:
# Cityを軸に、birth_yearの平均値を求める
attri_data_frame1.groupby("city")["birth_year"].mean()

city
Hokkaido    1996.0
Kyoto       1991.0
Osaka       1988.5
Tokyo       1986.0
Name: birth_year, dtype: float64

軸は複数でも設定可能で、次は、region, Cityを2軸に、birth_yearの平均値を求めています。

In [0]:
attri_data_frame1.groupby(["region","city"])["birth_year"].mean()

region    city    
Hokkaido  Hokkaido    1996.0
Kansai    Kyoto       1991.0
          Osaka       1988.5
Kanto     Tokyo       1986.0
Name: birth_year, dtype: float64

as_index=Falseにすることで、インデックスを設定しません。そのままテーブルとして扱いたいときに便利です。

In [0]:
attri_data_frame1.groupby(["region","city"],as_index=False)["birth_year"].mean()

,region,city,birth_year
0,Hokkaido,Hokkaido,1996.0
1,Kansai,Kyoto,1991.0
2,Kansai,Osaka,1988.5
3,Kanto,Tokyo,1986.0


最後に複数の異なる値を算出したい場合に、aggを使うと便利です。aggの引数には、関数名のリストを設定しています。以下は、カウント、平均、最大、最小を計算しています。

In [0]:
# 列に複数の関数を適応
functions = ['count','mean','max','min']
grouped_student_math_data1 = student_data_math.groupby(['sex','address'])
grouped_student_math_data1['age','G1'].agg(functions)

age                       G1                   
            count       mean max min count       mean max min
sex address                                                  
F   R          44  16.977273  19  15    44  10.295455  19   6
    U         164  16.664634  20  15   164  10.707317  18   4
M   R          44  17.113636  21  15    44  10.659091  18   3
    U         143  16.517483  22  15   143  11.405594  19   5

以上で、Pandasの章は終了です。説明を読んでいるだけではなかなかイメージを掴みにくい箇所もあったかもしれません。しかし、実際に「こんな感じでデータ加工や変換したいのになあ」と思ったときに、ここを参考にしてプログラミングをしてみてください。データ加工処理のニーズが出てきて、実際に使うことで一層理解が進む箇所かもしれません。また、ここで紹介したテクニックはほんの一部です。この他にも色々なデータ処理・加工方法があるので、以下の参考文献等を読んで、手を動かして実行してみてください。

>**[やってみよう]**

>ここで扱った例題の軸以外にも、対象データに対していろいろな軸で集計をしてみましょう。

>[参考文献]

>『Pythonによるデータ分析入門 ―NumPy、pandasを使ったデータ処理』（Wes McKinney (著), 小林 儀匡 (翻訳), 鈴木 宏尚 (翻訳), 瀬戸山 雅人  (翻訳)、オライリージャパン）

データ分析は、データの前処理が8割だとよく言われます。世の中には実に様々な形式のデータが存在し、それらを整えるだけでも大変な作業です。テクニックも重要ですが、それらに対してどのように対処していくのか戦略を立てることも重要です。以下の文献等も参考になりますので、ぜひ目を通してみてください。

>[参考文献]

>『バッドデータハンドブック』（磯蘭水等訳、オイラリージャパン）

#### <練習問題 1>

先ほど使用した「student-mat.csv」を使って「student-mat.csv」を使って、pandasの集計処理してみましょう。まずは、schoolを軸にして、G1の平均点をそれぞれ求めてみましょう。

#### <練習問題 2>

次は、schoolと性別を軸にして、G1,G2,G3の平均点をそれぞれ求めてみましょう。

#### <練習問題 3>

次は、schoolと性別を軸にして、G1,G2,G3の最大値、最小値をまとめて算出してみましょう。

***

## 6.2 欠損データと異常値の取り扱いの基礎
ゴール：欠損データと異常値に対する基本的な対処方法を知る

### 6.2.1 欠損データの扱い方
キーワード：リストワイズ削除、ペアワイズ削除、単一代入（平均値代入法）、最尤法、回帰代入法、スプライン補間

データを扱っていると必ずといっていいほど、欠損しているデータや異常値データの存在があります。それぞれ深い分野ですが、ここでは基礎の基礎レベルで欠損データや異常データについての判定や扱い方について学ぶことにします。もっと深く学びたい方は、ぜひ参考文献を読んでください。

>[参考文献]

>『欠損データの統計科学』（高井啓二等、岩波書店）

>『データ分析のプロセス』の第3章（福島真太郎、共立出版）

まずは、欠損データの取り扱いについてです。データの欠損は、入力忘れ、無回答、システム上の問題など様々な要因があります。「ない」データについては、無視をするのがいいのか、除外をするのがいいのか、最もらしい値を入れるのがいいのか、それが問題です。アプローチによっては、大きなバイアスのある結果を与え、誤った意思決定につながり、大きな損失につながる可能性もあります。慎重に扱っていきましょう。

以下では、欠損データに対して、削除や、0や直前の数字、平均値等で穴埋めをしています。ここでは、これらの単純な方法のみ紹介しますが、他の方法には最尤推定法で推定したり、回帰代入やScipyで実施したスプライン補間などがあります。注意が必要なのは、これらの方法がバイアスを生む可能性があることです。深く学びたい方はぜひ上で紹介した参考文献などを読んで、欠損データを埋める方法への理解を深めてください。

In [0]:
# データの準備
import numpy as np
from numpy import nan as NA
import pandas as pd


sample_data_frame = pd.DataFrame(np.random.rand(10,4))

# NAにする
sample_data_frame.ix[1,0] = NA
sample_data_frame.ix[2:3,2] = NA
sample_data_frame.ix[5:,3] = NA

用意したデータは以下になります。

In [0]:
sample_data_frame

,0,1,2,3
0,0.782601,0.510081,0.066241,0.051084
1,NaN,0.588928,0.481636,0.989737
2,0.770749,0.537386,NaN,0.415799
3,0.189696,0.288617,NaN,0.759283
4,0.661668,0.991183,0.223083,0.562561
5,0.497713,0.955180,0.301120,NaN
6,0.210177,0.643425,0.420065,NaN
7,0.089945,0.567024,0.692832,NaN
8,0.799016,0.940762,0.661627,NaN
9,0.616832,0.158643,0.091754,NaN


NaNがある行はすべて取り除くのは、dropnaを使います。これを**リストワイズ削除**といいます。以下は、先ほどのデータにおいて、全てのカラムにデータがある行だけ表示しています。NaNがある行は表示されません。

In [0]:
sample_data_frame.dropna()

,0,1,2,3
0,0.465470,0.742543,0.618442,0.009381
4,0.227128,0.238948,0.983770,0.940053


一方、リストワイズ削除では元々10行あったデータが極端に少なくなって2行になり、データが全く使えないという状況を考えましょう。このとき、欠損している列のデータを無視して、利用可能なデータのみ（例：列の0番目と1番目のみ存在）を使う方法があります。これを**ペアワイズ削除**といいます。

In [0]:
sample_data_frame[[0,1]].dropna()

,0,1
0,0.465470,0.742543
2,0.596530,0.862047
3,0.450267,0.721333
4,0.227128,0.238948
5,0.255822,0.719017
6,0.177791,0.659463
7,0.327858,0.780492
8,0.648819,0.520847
9,0.181585,0.427165


他の処理についても紹介します。次の処理はfillna(値)で、NaNになっている箇所をある値で埋めます。以下は、先ほどのデータでNaNになっているところを0で置き換えています。

In [0]:
sample_data_frame.fillna(0)

,0,1,2,3
0,0.465470,0.742543,0.618442,0.009381
1,0.000000,0.524610,0.369549,0.243735
2,0.596530,0.862047,0.000000,0.624683
3,0.450267,0.721333,0.000000,0.920845
4,0.227128,0.238948,0.983770,0.940053
5,0.255822,0.719017,0.661478,0.000000
6,0.177791,0.659463,0.254689,0.000000
7,0.327858,0.780492,0.409246,0.000000
8,0.648819,0.520847,0.730173,0.000000
9,0.181585,0.427165,0.659314,0.000000


methodを指定することで、前の値で埋めてくれます。

In [0]:
sample_data_frame.fillna(method="ffill")

,0,1,2,3
0,0.465470,0.742543,0.618442,0.009381
1,0.465470,0.524610,0.369549,0.243735
2,0.596530,0.862047,0.369549,0.624683
3,0.450267,0.721333,0.369549,0.920845
4,0.227128,0.238948,0.983770,0.940053
5,0.255822,0.719017,0.661478,0.940053
6,0.177791,0.659463,0.254689,0.940053
7,0.327858,0.780492,0.409246,0.940053
8,0.648819,0.520847,0.730173,0.940053
9,0.181585,0.427165,0.659314,0.940053


他、平均値でも穴埋めすることができます。これを**平均値代入法**といいます。

In [0]:
# 各カラムの平均値(確認用)
sample_data_frame.mean()

0    0.370141
1    0.619646
2    0.585833
3    0.547739
dtype: float64

In [0]:
sample_data_frame.fillna(sample_data_frame.mean())

,0,1,2,3
0,0.465470,0.742543,0.618442,0.009381
1,0.370141,0.524610,0.369549,0.243735
2,0.596530,0.862047,0.585833,0.624683
3,0.450267,0.721333,0.585833,0.920845
4,0.227128,0.238948,0.983770,0.940053
5,0.255822,0.719017,0.661478,0.547739
6,0.177791,0.659463,0.254689,0.547739
7,0.327858,0.780492,0.409246,0.547739
8,0.648819,0.520847,0.730173,0.547739
9,0.181585,0.427165,0.659314,0.547739


他にも色々とオプションがあるので、?sample_data_frame.fillna等で調べてみてください。

欠損データについて、今回はサンプルデータにおいて、一定の値を機械的に置換しました。ただし、これらの方法はいつも使えるというわけではありません。データの状況、背景等を考え、適切に対処することが重要です。

#### <練習問題 1>

以下のデータに対して、1列でもNaNがある場合は削除し、その結果を表示してください。

In [0]:
# データの準備
import numpy as np
from numpy import nan as NA
import pandas as pd


sample_data_frame2 = pd.DataFrame(np.random.rand(15,6))

# NAにする
sample_data_frame2.ix[2,0] = NA
sample_data_frame2.ix[5:8,2] = NA
sample_data_frame2.ix[7:9,3] = NA
sample_data_frame2.ix[10,5] = NA


sample_data_frame2

/Users/higuchiitto/.pyenv/versions/anaconda3-2.5.0/lib/python3.5/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  # Remove the CWD from sys.path while we load stuff.


,0,1,2,3,4,5
0,0.355722,0.473385,0.290236,0.489532,0.591536,0.087702
1,0.371207,0.715934,0.285595,0.023505,0.028515,0.299243
2,NaN,0.592995,0.901844,0.189380,0.888377,0.966973
3,0.419518,0.075851,0.440993,0.496051,0.294157,0.837886
4,0.853207,0.652506,0.993277,0.125946,0.254773,0.257159
5,0.922825,0.592044,NaN,0.356279,0.456573,0.628271
6,0.331692,0.959519,NaN,0.472897,0.753148,0.902748
7,0.201707,0.622600,NaN,NaN,0.835892,0.678096
8,0.123516,0.890916,NaN,NaN,0.142478,0.712876
9,0.220476,0.858546,0.504977,NaN,0.039185,0.809908


#### <練習問題 2>

上記で準備したデータに対して、NaNを0で埋めてください。

#### <練習問題 3>

上記で準備したデータに対して、NaNをそれぞれの列の平均値で埋めてください。

### 6.2.2 異常データの扱い方
キーワード：異常値、箱ひげ図、パーセンタイル、VaR(Value At Risk)

異常値（外れ値）データの扱いは、そのままにして何もしないのか、異常値を除去するか、もっともらしい値に入れかえて使うかが問題になります。そもそも異常値とは一体何でしょうか。実は、統一的な見解というものはなく、そのデータを扱うアナリストや意思決定者が判断することもあります。ビジネスの現場では、不正アクセスのパターン（セキュリティ分野）や機械の故障、金融リスク管理（VaR）など、様々な分野で使われており、それぞれ色々な方法でアプローチされています。

異常値検出のアプローチには、単純には箱ひげ図などを書いて、あるパーセンタイル以上のデータを異常値としてみなす方法、正規分布を利用する方法、データの空間的な近さに基づく方法などがあります。他には以降の章で学ぶ機械学習（教師なし学習も含む）を用いた方法があります。

ここでは特に練習問題はありませんが、興味のある方はぜひ以下の参考文献などで学んでください。

>[参考文献]

>『入門 機械学習による異常検知―Rによる実践ガイド』（井手剛、コロナ社）

>『異常検知と変化検知』（井手剛等、講談社）


>[参考URL]

>http://qiita.com/GushiSnow/items/f032806cfa8cec046318

>http://qiita.com/shopetan/items/ceb7744facc21c3881d2

>http://www.slideshare.net/shoheihido/fit2012

また、異常値の分野に関連して、極端な値を研究する極値統計学という分野もあります。データの中で大きな値をとる極値データの挙動について、様々な研究がなされており、稀ではありますがそれが起きれば非常に大きな影響を及ぼす現象（自然現象、災害など）を研究します。気象学だけではなく、ファイナンスや情報通信の分野でも応用されているようですので、興味のある方は以下の参考文献等見ながら、調べてみてください。

>[参考文献]

>『極値統計学 (ISMシリーズ:進化する統計数理)』（高橋 倫也 (著),志村 隆彰 (著)、近代科学社）

***

## 6.3 総合問題

### 6.3.1 総合問題1
以前使用した「student-mat.csv」を使って、以下の問いに答えてください。

(1) 上記のデータに対して、年齢×性別でG1の平均点を算出し、縦軸が年齢、横軸が性別となるような表（テーブル）を作成しましょう。

(2) (1)で表示した結果テーブルについて、NAになっている行(レコード)を全て削除した結果を表示しましょう。